# Installation of required packages and libraries

In [ ]:
# The versions are important otherwise the package dependencies will likely fail
print('\nINSTALLING REQUIRED PACKAGES...')
!pip install --upgrade pip==21.1.3

!pip install --upgrade setuptools==59.5.0
# CPU version of pytorch has smaller footprint - see installation instructions in
# pytorch documentation - https://pytorch.org/get-started/locally/
#!python -m pip install mitmproxy
#!pip install torch==1.10.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip install torch==1.10.2 -f https://download.pytorch.org/whl/cpu/torch_stable.html

# AutoGluon(2020): This popular AutoML open-source toolkit developed by AWS helps in getting a strong predictive performance 
# in various machine learning and deep learning models on text, image, and tabular data.
!pip install autogluon

!pip install --upgrade scikit-learn==1.0.0

print('\nREQUIRED PACKAGES INSTALLED')

In [ ]:
# For visualisation of plots
!pip install seaborn

In [ ]:
# For writing/saving structural alert images to excel file
!pip install xlsxwriter

# Automated machine learning model training and evaluation

In [ ]:
# Author: Marcus Wei How Wang
# 5 May 2022

#======================================================================================#
import pandas as pd
import numpy as np

import os
import os.path
from os.path import isfile

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import make_scorer

from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

import pickle
from random import randrange

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.metrics import make_scorer

#======================================================================================#
# This function gets the Morgan fingeprint given a SMILES dataframe and specified fingerprint parameters
def get_Morgan_fingerprint(smiles,nBits,fingerprint_radius):
    
    '''smiles dataframe'''
    error_idx_ls =[]
    error_idx_ls.clear()
    
    rdkit_molecules=[Chem.MolFromSmiles(x) for x in smiles]
    print(len(rdkit_molecules))
    rdkit_fingerprint=[]
    count = 0
    fingerprint_length = int(nBits)
    #print(rdkit_molecules[:1])
    for mol in rdkit_molecules:
        # if count % 1000 == 0:
        #     print('Now fingerprinting {} of {}'.format(count,len(rdkit_molecules)))
        bit_info={}
        try:
            fp=rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=fingerprint_radius, nBits=fingerprint_length,
                                                          bitInfo=bit_info)
        except:
            # get indexes of errors with errors
            error_idx_ls.append(rdkit_molecules.index(mol))

        rdkit_fingerprint.append(fp)
        count += 1
        
    fingerprint_df=pd.DataFrame([np.array(list(x)).astype(int) for x in rdkit_fingerprint])
    #fingerprint_df = pd.DataFrame(rdkit_fingerprint,columns=['BV'])
    
    return fingerprint_df, error_idx_ls

def get_MACCS_fingerprint(smiles):
    
    
    
    return fingerprint_df


# This function is expecting a input csv with two columns 'SMILES' and 'Binary Activity'
def split_train_test(csv,test_ratio,train_save,test_save,overwrite):
    
    # Read input and shuffle randomly
    input_df = pd.read_csv(csv)
    print(input_df)
    input_df = input_df.sample(frac=1)
    input_np = input_df.to_numpy()
    
    # split df randomly according to specified ratio
    train_df, test_df = train_test_split(input_np, test_size=test_ratio)
    
    train_df = pd.DataFrame(train_df,columns=['SMILES','Binary Activity'])
    test_df = pd.DataFrame(test_df,columns=['SMILES','Binary Activity'])
    
    # Save files
    # Ovewrite existing file present in folder
#     print(isfile(train_save))
#     print(isfile(test_save))
    if overwrite == False and isfile(train_save) == True:
        train_save = train_save[:-4] + str(randrange(100)) + train_save[-4:]
        train_df.to_csv(train_save)        
    else:
        train_df.to_csv(train_save)
    
    if overwrite == False and isfile(test_save) == True:
        test_save = test_save[:-4] + str(randrange(100)) + test_save[-4:]
        test_df.to_csv(test_save)        
    else:
        test_df.to_csv(test_save)
    
    
    return train_save,test_save

def most_probable_class(column_list):
    col_val = set(column_list)
    most_prob_class_count = 0
    most_prob_class = 0
    for ele in col_val:
        class_count = column_list.count(ele)
        if class_count > most_prob_class_count:
            most_prob_class_count = class_count
            most_prob_class = ele
            
    return most_prob_class


def calc_baseline_acc(df_column_list):
    
    # Baseline accuracy is the accuracy when all the predicted classes are the most probable class
    baseline_class = most_probable_class(df_column_list)
    baseline_acc = df_column_list.count(baseline_class) / len(df_column_list) * 100
    
    print('Baseline accuracy:')
    print(baseline_acc)
    
    return baseline_acc

def acc(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return (tp+tn)/(tp+fp+fn+tn)

def sensitivity(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return (tp)/(tp+fn)

def specificity(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return (tn)/(fp+tn)

def tp_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return tp

def fn_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    # Needs the - sign since predictor.leaderboard flips the sign
    return -fn  

def fp_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    # Needs the - sign since predictor.leaderboard flips the sign
    return -fp  

def tn_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return tn  


def create_AutoGluon_extra_metrics(y_test,y_pred):

    # Score functions need to have the definition:
    # score_func(y, y_pred, **kwargs)
    # Otherwise the calculations of the metrics will fail
    
    metrics_ls = []
    metrics_ls.clear()
    
    # Calculate confusion matrix for custom metrics
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    MCC = matthews_corrcoef(y_test, y_pred)
    
      
    # Define custom scoring functions to pass to predictor.leaderboard()
    ag_accuracy_scorer = make_scorer(name='Acc',
                                 score_func=acc,
                                 optimum=1,
                                 greater_is_better=True)
    
    ag_mcc_scorer = make_scorer(name='MCC',
                                 score_func=matthews_corrcoef,
                                 optimum=1,
                                 greater_is_better=True)
    
    ag_sensitivity_scorer = make_scorer(name='SE',
                                 score_func=sensitivity,
                                 optimum=1,
                                 greater_is_better=True)

    ag_specificity_scorer = make_scorer(name='SP',
                                 score_func=specificity,
                                 optimum=1,
                                 greater_is_better=True)
    
  
    tn_scorer = make_scorer(name='TN',score_func=tn_func,greater_is_better=True)

    
      
    fp_scorer = make_scorer(name='FP',score_func=fp_func,greater_is_better=False)

    
      
    fn_scorer = make_scorer(name='FN',score_func=fn_func,greater_is_better=False)

    

    tp_scorer = make_scorer(name='TP',score_func=tp_func,greater_is_better=True)

    
    # Append all metrics to list
    metrics_ls.append(tp_scorer)
    metrics_ls.extend((fp_scorer, fn_scorer, tn_scorer, ag_sensitivity_scorer,
                      ag_specificity_scorer, ag_accuracy_scorer, ag_mcc_scorer))
#     print(metrics_ls)
    return metrics_ls
#======================================================================================#
# The function is expecting a training and test set with SMILES and Binary Activity (2 columns in total)
def AutoGluon(training_set,test_set,save_path,model_results,model_name):
    print('\nSETTING UP DATA FOR MODEL...')
    dat_train = pd.read_csv(training_set, sep=',')
    dat_train_smi = dat_train['SMILES'].tolist()
    print('\ndat_train no. of SMILES:')
    print(len(dat_train_smi))

    y_train = dat_train[['Binary Activity']]
    X_train,error_idx_ls = get_Morgan_fingerprint(dat_train_smi,2048,2)

    y_train = y_train.drop(error_idx_ls)
    X_train = X_train.drop(error_idx_ls)

    dat_test = pd.read_csv(test_set, sep=',')
    dat_test_smi = dat_test['SMILES'].tolist()
    print('\ndat_test no. of SMILES:')
    print(len(dat_test_smi))

    y_test = dat_test[['Binary Activity']]
    X_test,error_idx_ls = get_Morgan_fingerprint(dat_test_smi,2048,2)

    # Drop rows with errors
    y_test = y_test.drop(error_idx_ls)
    X_test = X_test.drop(error_idx_ls)

    print(X_train)
    print(y_train)

    print('\nDATA SET UP FOR MODEL')
    
    print('\nPREPARING AND TRAINING MODELS...')
    # The dataframes need to be converted to the object used byu the AutoGluon package
    label = 'Binary Activity'
    train_data = TabularDataset(pd.concat([X_train,y_train],axis=1))
    
    # This the maximum time limit in seconds for each set of models
    # Essentially, this means the models will take an estimated (13 * time_limit * (num_stack_levels +1)) 
    # amount of seconds to train. Note training time is also affected by the preset quality
    time_limit = 60 * 60
    
    # Define metrics for validation here
    # If using a different metric other than accuracy, note that the preset below needs to be changed as well
    # Otherwise, the model will focus on acc
    metric = 'accuracy'
    ag_sensitivity_scorer = make_scorer(name='SE',
                             score_func=sensitivity,
                             optimum=1,
                             greater_is_better=True)

    ag_specificity_scorer = make_scorer(name='SP',
                                 score_func=specificity,
                                 optimum=1,
                                 greater_is_better=True)
    
    # Best quality improves performance but takes longer to run. Medium_quality is the default setting.
    # For more details see the AutoGluon package documentation
#     preset = 'best_quality'
    preset = 'medium_quality'
    
    # add hyperparameter_tune_kwargs if necessary
#     num_trials = 25  # try at most n different hyperparameter configurations for each type of model
#     search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

#     hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
#         'num_trials': num_trials,
#         'scheduler' : 'local',
#         'searcher': search_strategy,
#     }
    
    
    # Model ensembling with stacking/bagging
    # According to the documentation, "beyond hyperparameter-tuning with a correctly-specified evaluation metric,
    # two other methods to boost predictive performance are bagging and stack-ensembling. 
    # You’ll often see performance improve if you specify num_bag_folds = 5-10, 
    # num_stack_levels = 1-3 in the call to fit(), but this will increase training times and memory/disk usage."
    # By default no hyperparameter optimisation is done ie. when hyperparameter_tune_kwargs is not specified
    predictor = TabularPredictor(label, eval_metric=ag_sensitivity_scorer,path=model_name).fit(train_data,num_bag_folds=5, 
                                                                num_bag_sets=1, num_stack_levels=1, 
                                                                time_limit=time_limit,
                                                                presets=preset
#                                                                ,hyperparameter_tune_kwargs=hyperparameter_tune_kwargs
                                                               )
                                                               
    print('\nMODELS TRAINED')
    
    print('\nEVALUATING MODELS...')
    test_data = TabularDataset(pd.concat([X_test,y_test],axis=1))
    y_test = test_data[label]  # values to predict
    X_test = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
    X_test.head()
    
    y_pred = predictor.predict(X_test)
    #print("Predictions:  \n", y_pred)
    #perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
    
    #print("TRUE VALUES:  \n", y_test)
    y_test = y_test.tolist()
    y_pred = y_pred.tolist()
    
    add_metrics = create_AutoGluon_extra_metrics(y_test,y_pred)

    results_df = predictor.leaderboard(test_data, silent=True,
                                       extra_metrics=add_metrics
                                      )
    results_df.to_csv(model_results)
    print(results_df)
    print('\nMODELS EVALUATED')   
    
    return
#==============================================================================================#
#==============================================================================================#
# Set filepaths for functions

# Filepath of csv containing two columsn 'SMILES' and 'Binary Activity'
root_desc = 'V2.0.1 Unified data max SP medium'
root = 'C:/Users/mwhw3/Desktop/DART project/'
input_path = root + root_desc + '.csv'

total_runs = 5

overall_save = root + 'AutoML models combined results/' + root_desc + ' ' + str(total_runs) +' runs.csv'
#overall_save = root + 'AutoML models combined results/' + root_desc + ' all Unified data external prediction' + \
str(total_runs) +' runs.csv'

for run in range(1,total_runs+1):
    
    print('\n#=========================================================================================#')
    print('#=========================================================================================#')
    print('\n                               NOW PERFORMING RUN {}\n'.format(run))
    print('#=========================================================================================#')
    print('#=========================================================================================#')
    
    desc = root_desc + ' ' + str(run)
    check_path = root + 'AutoML models/' + desc + '/' + 'Data/'
    if os.path.exists(check_path)== False:
        os.makedirs(check_path)
        
    # Save locations for training and test sets                              
    train_save = root + 'AutoML models/' + desc + '/' + 'Data/' + desc + ' CSV train.csv'
    test_save = root + 'AutoML models/' + desc + '/' + 'Data/' + desc + ' CSV test.csv'

    # Save location for model results
    model_name = root + 'AutoML models/' + desc + '/' + 'Models/'
    if os.path.exists(model_name)== False:
        os.makedirs(model_name)
    check_path = root + 'AutoML models/' + desc + '/' + 'Results/'
    if os.path.exists(check_path)== False:
        os.makedirs(check_path)
    model_results = root + 'AutoML models/' + desc + '/' + 'Results/' + desc + ' model results.csv'

    # The AutoGluon package creates a folder if one is not present
    # Otherwise it will save in the specified filepath
    save_path = root + 'AutoML models/' + desc + '/'                            
    #==============================================================================================#                             
    # Main code
    #input_csv = pd.read_csv(input_path)
    training_set,test_set = split_train_test(input_path,0.2,train_save,test_save,True)                              

    # If running AutoGluon model code                         
    AutoGluon(training_set,test_set,save_path,model_results,model_name)
    
print('\n#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')
print('\n                             ALL MODELS TRAINED AND EVALUATED\n'                          )
print('#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')

#==============================================================================================#
#==============================================================================================#    
# Combine results for models across all runs    

# Filepath of csv containing two columsn 'SMILES' and 'Binary Activity'
#col_ls = ['TP','FP','FN','TN','SE','SP','Acc','MCC']
col_ls = ['model','SE','SP','Acc','MCC']

result_dict = {}
for run in range(1,total_runs+1):
    desc = root_desc + ' ' + str(run)
    model_results = root + 'AutoML models/' + desc + '/' + 'Results/' + desc + ' model results.csv'
    results_df = pd.read_csv(model_results)
    results_df = results_df[col_ls]
    results_name = str(results_df) + str(run)
    result_dict[results_name] = results_df
    
combined_df = pd.concat(result_dict.values())
#print(combined_df)
mean_df = combined_df.groupby(by=['model']).mean()

print('\n#=========================================================================================#')
print('\n                                 DATAFRAME OF MEANS\n'                                      )
print('#=========================================================================================#\n')

mean_df['SE'] = 100 * mean_df['SE']
mean_df['SP'] = 100 * mean_df['SP']
mean_df['Acc'] = 100 * mean_df['Acc']

mean_df = mean_df.round({'SE': 1,'SP': 1,'Acc': 1,'MCC': 3}).astype(str)
print(mean_df)

std_df = combined_df.groupby(by=['model']).agg(np.std)
print('\n#=========================================================================================#')
print('\n                                 DATAFRAME OF STD\n'                                        )
print('#=========================================================================================#\n')

std_df['SE'] = 100 * std_df['SE']
std_df['SP'] = 100 * std_df['SP']
std_df['Acc'] = 100 * std_df['Acc']

std_df = std_df.round({'SE': 1,'SP': 1,'Acc': 1,'MCC': 3}).astype(str)
print(std_df)

overall_df = mean_df

overall_df['SE'] = mean_df['SE'] + ' '+ str(u"\u00B1") + ' ' + std_df['SE']
overall_df['SP'] = mean_df['SP'] + ' '+ str(u"\u00B1") + ' ' + std_df['SP']
overall_df['Acc'] = mean_df['Acc'] + ' '+ str(u"\u00B1") + ' ' + std_df['Acc']
overall_df['MCC'] = mean_df['MCC'] + ' '+ str(u"\u00B1") + ' ' + std_df['MCC']

print('\n#=========================================================================================#')
print('\n                                 OVERALL DATAFRAME\n'                                        )
print('#=========================================================================================#\n')

overall_df.to_csv(overall_save)
print(overall_df)


print('\nFINISHED')

# For visualising feature space with PCA or t-SNE

In [ ]:
# Code for visualsiing feature space
# t-SNE is used as it known to offer a better visual representation in the 2D plot as compared to PCA

import time

import pandas as pd
import numpy as np

import sklearn
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import seaborn as sns

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

#==============================================================================================#
#==============================================================================================#


# This function gets the Morgan fingeprint given a SMILES dataframe and specified fingerprint parameters
def get_Morgan_fingerprint(smiles,nBits,fingerprint_radius):
    
    '''smiles dataframe'''
    error_idx_ls =[]
    error_idx_ls.clear()
    
    rdkit_molecules=[Chem.MolFromSmiles(x) for x in smiles]
    print(len(rdkit_molecules))
    rdkit_fingerprint=[]
    count = 0
    fingerprint_length = int(nBits)
    #print(rdkit_molecules[:1])
    for mol in rdkit_molecules:
        # if count % 1000 == 0:
        #     print('Now fingerprinting {} of {}'.format(count,len(rdkit_molecules)))
        bit_info={}
        try:
            fp=rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=fingerprint_radius, nBits=fingerprint_length,
                                                          bitInfo=bit_info)
        except:
            # get indexes of errors with errors
            error_idx_ls.append(rdkit_molecules.index(mol))

        rdkit_fingerprint.append(fp)
        count += 1
        
    fingerprint_df=pd.DataFrame([np.array(list(x)).astype(int) for x in rdkit_fingerprint])
    #fingerprint_df = pd.DataFrame(rdkit_fingerprint,columns=['BV'])
    
    return fingerprint_df, error_idx_ls


#==============================================================================================#
#==============================================================================================#
# Functions
def input_to_tsne_plot(input_df,filename,perplex,iterations):
    # Data should only contain the feature columns ie. no label column
    print('\n#=========================================================================================#')
    print('\n                                      RUNNING t-SNE for                                       \n')
    print('                             ' + str(filename) + '                             ')
    print('\n')
    print('#=========================================================================================#\n')
    input_df['SMILES'].replace('', np.nan, inplace=True)
    input_df.dropna(subset=['SMILES'], inplace=True)
    input_df = input_df.drop_duplicates(subset=['SMILES'], keep='first', inplace=False)

    to_concat = input_df
    print(to_concat)
    
    input_df_smi = input_df['SMILES'].tolist()
    print('\ninput_df no. of SMILES:')
    print(len(input_df_smi))

    plot_data,error_idx_ls = get_Morgan_fingerprint(input_df_smi,2048,2)
    plot_data = plot_data.drop(error_idx_ls)
    to_concat = to_concat.drop(error_idx_ls)
    to_concat = to_concat.reset_index(drop=True)
    print(plot_data.head())

    # t-SNE
    time_start = time.time()
    # This gives an ndarray of n samples and 2 columns
    tsne = TSNE(n_components=2, perplexity=perplex,n_iter=iterations,verbose=1)
    tsne_results = tsne.fit_transform(plot_data)
    print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))
    
    return tsne_results,to_concat

# # PLotting the t-SNE results
# # This gives the x-axis
# plot_data['tsne-2d-one'] = tsne_results[:,0]
# # This gives the y-axis
# plot_data['tsne-2d-two'] = tsne_results[:,1]

def input_to_pca_plot(input_df,filename):
    # Data should only contain the feature columns ie. no label column
    print('\n#=========================================================================================#')
    print('\n                                      RUNNING PCA for                                       \n')
    print('                             ' + str(filename) + '                             ')
    print('\n')
    print('#=========================================================================================#\n')
    input_df['SMILES'].replace('', np.nan, inplace=True)
    input_df.dropna(subset=['SMILES'], inplace=True)
    input_df = input_df.drop_duplicates(subset=['SMILES'], keep='first', inplace=False)
    to_concat = input_df
    input_df_smi = input_df['SMILES'].tolist()
    print('\ninput_df no. of SMILES:')
    print(len(input_df_smi))

    plot_data,error_idx_ls = get_Morgan_fingerprint(input_df_smi,2048,2)
    plot_data = plot_data.drop(error_idx_ls)
    to_concat = to_concat.drop(error_idx_ls)
    to_concat = to_concat.reset_index(drop=True)
    print(plot_data.head())

    # t-SNE
    time_start = time.time()
    # This gives an ndarray of n samples and 2 columns
    pca = PCA(n_components = 2)
    pca_results = pca.fit_transform(plot_data)
    print('\nPercentage variance explained: {}'.format(pca.explained_variance_))
    print('PCA done! Time elapsed: {} seconds'.format(time.time()-time_start))
    
    return pca_results,to_concat

def tsne_main(root_desc,perplex,iterations):
    tsne_dict={}
    for filename in root_desc:
        index = root_desc.index(filename)

        if filename == 'external test set 1 no outliers':
            input_path = root + 'Outliers/' + filename + '.csv'

        else:
            input_path = root + filename + '.csv'

        input_df = pd.read_csv(input_path, sep=',')

        tsne_results,to_concat = input_to_tsne_plot(input_df,filename,perplex,iterations)

        tsne_df = pd.DataFrame(tsne_results, columns = ['tsne-2d-one','tsne-2d-two'])
        print(tsne_df.head())

        # Assign label by testing type for plotting later
        if filename == 'V2.0.1 in vivo data':
            new_col = 'in vivo'
        if filename == 'V2.0.1 in vitro data':
            new_col = 'in vitro'              
        
        # Assign label by toxicity endpoint for plotting later
        if filename == 'V2.0.1 Developmental non-toxicants':
            new_col = 'Developmental non-toxicants'
        if filename == 'V2.0.1 Developmental toxicants':
            new_col = 'Developmental toxicants'
        if filename == 'V2.0.1 Reproductive non-toxicants':
            new_col = 'Reproductive non-toxicants'
        if filename == 'V2.0.1 Reproductive toxicants':
            new_col = 'Reproductive toxicants'
              
        
        # Assign label by source for plotting later
        if filename == 'V2.0.1 Stemina data':
            new_col = 'Stemina'

        if filename == 'V2.0.1 Unified data no Stemina data':
            new_col = 'Unified data no Stemina' 

        if filename == 'V2.0.1 VEGA data':
            new_col = 'VEGA'

        if filename == 'V2.0.1 DEREK data':
            new_col = 'DEREK'
            
        if filename == 'V2.0.1 All other data':
            new_col = 'All other data'
            
        if filename == 'V2.0.1 Challa data':
            new_col = 'Challa et al.'

        if filename == 'V2.0.1 Prenatal dev tox':
            new_col = 'Prenatal dev tox'

        if filename == 'V2.0.1 ML data':
            new_col = 'Reproductive tox Feng et al.'

        if filename == 'V2.0.1 external test set':
            new_col = 'External test set'

        if filename == 'external test set no outliers':
            new_col = 'External test set (no outliers)'



        tsne_df = tsne_df.assign(dataset = new_col)    

        tsne_dict[new_col] = tsne_df
    
    return tsne_dict,to_concat

def pca_main(root_desc):
    pca_dict={}
    for filename in root_desc:
        index = root_desc.index(filename)

        if filename == 'external test set no outliers':
            input_path = root + 'Outliers/' + filename + '.csv'

        else:
            input_path = root + filename + '.csv'

        input_df = pd.read_csv(input_path, sep=',')

        pca_results,to_concat = input_to_pca_plot(input_df,filename)

        pca_df = pd.DataFrame(pca_results, columns = ['pca-one','pca-two'])
        print(pca_df.head())

        
        # Assign label by testing type for plotting later
        if filename == 'V2.0.1 in vivo data':
            new_col = 'in vivo'
        if filename == 'V2.0.1 in vitro data':
            new_col = 'in vitro'       
        
        # Assign label by toxicity endpoint for plotting later
        if filename == 'V2.0.1 Developmental non-toxicants':
            new_col = 'Developmental non-toxicants'
        if filename == 'V2.0.1 Developmental toxicants':
            new_col = 'Developmental toxicants'
        if filename == 'V2.0.1 Reproductive non-toxicants':
            new_col = 'Reproductive non-toxicants'
        if filename == 'V2.0.1 Reproductive toxicants':
            new_col = 'Reproductive toxicants'
              
        
        # Assign label by source for plotting later
        if filename == 'V2.0.1 Stemina data':
            new_col = 'Stemina'

        if filename == 'V2.0.1 Unified data no Stemina data':
            new_col = 'Unified data no Stemina' 

        if filename == 'V2.0.1 VEGA data':
            new_col = 'VEGA'

        if filename == 'V2.0.1 DEREK data':
            new_col = 'DEREK'
            
        if filename == 'V2.0.1 All other data':
            new_col = 'All other data'
            
        if filename == 'V2.0.1 Challa data':
            new_col = 'Challa et al.'

        if filename == 'V2.0.1 Prenatal dev tox':
            new_col = 'Prenatal dev tox'

        if filename == 'V2.0.1 ML data':
            new_col = 'Reproductive tox Feng et al.'

        if filename == 'V2.0.1 external test set':
            new_col = 'External test set'

        if filename == 'external test set no outliers':
            new_col = 'External test set (no outliers)'



        pca_df = pca_df.assign(dataset = new_col)    

        pca_dict[new_col] = pca_df
    
    return pca_dict,to_concat


# Input test_points as a dataframe with ['tsne-2d-one'] and ['tsne-2d-two'] as columns
def outlier_detection(query_point,test_points,threshold,points_in_circle):
    
    # Test if there are a certain number of test points within the circle of radius = threshold
    # for a given query point
    # Each point will have a x-value and a y-value   
    
    count = 0
    
    for n in range(0,len(test_points)):
        
        x = test_points.iloc[n]['tsne-2d-one']
        y = test_points.iloc[n]['tsne-2d-two']
        
        x_center = query_point['tsne-2d-one']
        y_center = query_point['tsne-2d-two']
        
        check = ((x - x_center)**2) + ((y - y_center)**2)
        
        if check <= (threshold**2):
            count = count + 1
    
    # function will return true if there are <= the specified no. of points in circle within the radius/threshold
    if count <= points_in_circle:
        return True
    
    else:
        return False

def calc_tsne_outliers(value):
    print('CALCULATING OUTLIERS...')
    query_df = tsne_dict['External test set 1']
    test_df = tsne_dict['Unified data no Stemina']

    outlier_ls = []
    outlier_ls.clear()

    # The threshold specified is the radius
    # TYhe values should be a reasonable distance on the t-SNE plot
    threshold = 25
    points_in_circle = 1

    for query in range(0,len(query_df)):

        if query % 10 == 0:
            print('\nNOW FINDING OUTLIERS FOR INDEX {}'.format(query))

        query_point = query_df.iloc[query] 

        if outlier_detection(query_point,test_df,threshold,points_in_circle) == True:
            outlier_ls.append(query)


    # Get outliers in external test set as df for easy access if necessary
    index = root_desc.index(filename)
    input_path = root + root_desc[1] + '.csv'
    input_df = pd.read_csv(input_path, sep=',')

    outlier_df = input_df.iloc[outlier_ls]
    print('\n')
    print('{} OUTLIERS WERE FOUND ON THIS RUN'.format(outlier_df))

    print('\n')
    print(outlier_df.head())
    
    outlier_df.to_csv(outlier_save)

    # save external test set without outliers
    ext_no_outlier_df = input_df.drop(outlier_ls)
    print(ext_no_outlier_df)
    ext_no_outlier_df.to_csv(ext_no_outlier)        
    
#==============================================================================================#
#==============================================================================================#
# Main code
# Filepaths of csv containing two columsn 'SMILES' and 'Binary Activity' to be input in
# root_desc as a list. This is for plotting t-SNE results of different datasets on the same plot

root = 'C:/Users/mwhw3/Desktop/DART project/'

# root_desc = ['V2.0.1 Stemina data']
# save_desc = 'Stemina data'

# root_desc = ['V2.0.1 Stemina data','V2.0.1 VEGA data','V2.0.1 DEREK data','V2.0.1 Challa data',
#              'V2.0.1 Prenatal dev tox', 'V2.0.1 ML data','V2.0.1 All other data']
# save_desc = 'Data by source'

# root_desc = ['V2.0.1 Developmental non-toxicants','V2.0.1 Developmental toxicants',
#               'V2.0.1 Reproductive non-toxicants','V2.0.1 Reproductive toxicants']
# save_desc = 'Data by toxicity endpoint'

root_desc = ['V2.0.1 in vivo data','V2.0.1 in vitro data']
save_desc = 'Data by testing type'


# Save location for resulting sns plot
plot_save_tsne = root + 'Feature plots/'+ save_desc + '_tsne.tiff'
plot_save_pca = root + 'Feature plots/'+ save_desc + '_pca.tiff'
plot_save_pca_facet = root + 'Feature plots/'+ save_desc + '_pca_facet.tiff'
#plot_save = root + 'SNS plots/'+ 'Compare Unified data with Stemina' + '.tiff'
#plot_save = root + 'SNS plots/'+ 'Stemina data only' + '.tiff'
#plot_save = root + 'SNS plots/'+ 'Predicting on external test set 1' + '.tiff'

# Change tsne parameters
# Note perplexity is the guess of the number of nearest neighbours per data point

# perplex_ls = [10]
perplex_ls = [10,20,40,60,80]
# iterations = 5000
iterations = 1000
#==================================================================================#  
# # Plot data from specified feature visualisation method on the same axis

# For plotting tsne
# for perplex in perplex_ls:
#     print('\nRUNNING tsne with perplexity:',perplex)
#     tsne_dict,to_concat = tsne_main(root_desc,perplex,iterations)
#     tsne_concatenated = pd.concat(tsne_dict.values(), sort=False, ignore_index=False)
#     tsne_concatenated.columns= ['tsne-2d-one','tsne-2d-two','dataset']
#     plt.figure(figsize=(16,10))
#     sns.scatterplot(x="tsne-2d-one", y="tsne-2d-two", data=tsne_concatenated,
#                     hue='dataset', palette="colorblind", legend="full", style='dataset')

#     plot_save_tsne = root + 'Feature plots/'+ save_desc + '_tsne_perplex' + str(perplex) + '.tiff'
#     plt.savefig(plot_save_tsne)
#     plt.show()

# For plotting PCA
pca_dict,to_concat = pca_main(root_desc)
pca_concatenated = pd.concat(pca_dict.values(), sort=False, ignore_index=False)  
pca_concatenated.columns= ['pca-one','pca-two','dataset']
plt.figure(figsize=(16,10))
sns.scatterplot(x="pca-one", y="pca-two", data=pca_concatenated,
                hue='dataset', palette="colorblind", legend="full", style='dataset')

# Get tick labels for facet plot
# Remove first and last tick label which are the plot boundaries 
xticks = list(plt.xticks()[0])
xticks = [int(x) for x in xticks]
xticks = xticks[1:-1]

yticks = list(plt.yticks()[0])
yticks = [int(y) for y in yticks]
yticks = yticks[1:-1]
print(xticks)
print(yticks)

#plt.savefig(plot_save_pca)
plt.show()

# Plot Facet grid for PCA plot to better visualise the global structure of the feature space
# This also enables the overlapping compounds to be visaulised better
plots_per_row = 2             # This controls how many plots appear in the Facet Grid plot per row
facet = sns.FacetGrid(pca_concatenated, col="dataset", col_wrap=plots_per_row, palette="colorblind", 
                      hue="dataset", height = 5)
facet = (facet.map(plt.scatter, "pca-one", "pca-two", s=10, edgecolor="w"))

facet.set(xticks=xticks, yticks=yticks)
plt.savefig(plot_save_pca_facet)
plt.show()

    
print('\nFINISHED')

# For loading all trained models and predicting on external test set

In [3]:
# Code for using trained models to predict on an external test set or external data

# Author: Marcus Wei How Wang
# 5 May 2022

#======================================================================================#
import pandas as pd
import numpy as np

import os
import os.path
from os.path import isfile

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import make_scorer

from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

import pickle
from random import randrange

from autogluon.tabular import TabularDataset, TabularPredictor

#======================================================================================#
# This function gets the Morgan fingeprint given a SMILES dataframe and specified fingerprint parameters
def get_Morgan_fingerprint(smiles,nBits,fingerprint_radius):
    
    '''smiles dataframe'''
    error_idx_ls =[]
    error_idx_ls.clear()
    
    rdkit_molecules=[Chem.MolFromSmiles(x) for x in smiles]
    print(len(rdkit_molecules))
    rdkit_fingerprint=[]
    count = 0
    fingerprint_length = int(nBits)
    #print(rdkit_molecules[:1])
    for mol in rdkit_molecules:
        # if count % 1000 == 0:
        #     print('Now fingerprinting {} of {}'.format(count,len(rdkit_molecules)))
        bit_info={}
        try:
            fp=rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=fingerprint_radius, nBits=fingerprint_length,
                                                          bitInfo=bit_info)
        except:
            # get indexes of errors with errors
            error_idx_ls.append(rdkit_molecules.index(mol))

        rdkit_fingerprint.append(fp)
        count += 1
        
    fingerprint_df=pd.DataFrame([np.array(list(x)).astype(int) for x in rdkit_fingerprint])
    #fingerprint_df = pd.DataFrame(rdkit_fingerprint,columns=['BV'])
    
    return fingerprint_df, error_idx_ls

def get_MACCS_fingerprint(smiles):
    
    
    
    return fingerprint_df


# This function is expecting a input csv with two columns 'SMILES' and 'Binary Activity'
def split_train_test(csv,test_ratio,train_save,test_save,overwrite):
    
    # Read input and shuffle randomly
    input_df = pd.read_csv(csv)
    print(input_df)
    input_df = input_df.sample(frac=1)
    input_np = input_df.to_numpy()
    
    # split df randomly according to specified ratio
    train_df, test_df = train_test_split(input_np, test_size=test_ratio)
    
    train_df = pd.DataFrame(train_df,columns=['SMILES','Binary Activity'])
    test_df = pd.DataFrame(test_df,columns=['SMILES','Binary Activity'])
    
    # Save files
    # Ovewrite existing file present in folder
#     print(isfile(train_save))
#     print(isfile(test_save))
    if overwrite == False and isfile(train_save) == True:
        train_save = train_save[:-4] + str(randrange(100)) + train_save[-4:]
        train_df.to_csv(train_save)        
    else:
        train_df.to_csv(train_save)
    
    if overwrite == False and isfile(test_save) == True:
        test_save = test_save[:-4] + str(randrange(100)) + test_save[-4:]
        test_df.to_csv(test_save)        
    else:
        test_df.to_csv(test_save)
    
    
    return train_save,test_save

def most_probable_class(column_list):
    col_val = set(column_list)
    most_prob_class_count = 0
    most_prob_class = 0
    for ele in col_val:
        class_count = column_list.count(ele)
        if class_count > most_prob_class_count:
            most_prob_class_count = class_count
            most_prob_class = ele
            
    return most_prob_class


def calc_baseline_acc(df_column_list):
    
    # Baseline accuracy is the accuracy when all the predicted classes are the most probable class
    baseline_class = most_probable_class(df_column_list)
    baseline_acc = df_column_list.count(baseline_class) / len(df_column_list) * 100
    
    print('Baseline accuracy:')
    print(baseline_acc)
    
    return baseline_acc

def acc(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return (tp+tn)/(tp+fp+fn+tn)

def sensitivity(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return (tp)/(tp+fn)

def specificity(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return (tn)/(fp+tn)

def tp_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return tp

def fn_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    # Needs the - sign since predictor.leaderboard flips the sign
    return -fn  

def fp_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    # Needs the - sign since predictor.leaderboard flips the sign
    return -fp  

def tn_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return tn  


def create_AutoGluon_extra_metrics(y_test,y_pred):
    from autogluon.core.metrics import make_scorer
    # Score functions need to have the definition:
    # score_func(y, y_pred, **kwargs)
    # Otherwise the calculations of the metrics will fail
    
    metrics_ls = []
    metrics_ls.clear()
    
    # Calculate confusion matrix for custom metrics
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    MCC = matthews_corrcoef(y_test, y_pred)
    
      
    # Define custom scoring functions to pass to predictor.leaderboard()
    ag_accuracy_scorer = make_scorer(name='Acc',
                                 score_func=acc,
                                 optimum=1,
                                 greater_is_better=True)
    
    ag_mcc_scorer = make_scorer(name='MCC',
                                 score_func=matthews_corrcoef,
                                 optimum=1,
                                 greater_is_better=True)
    
    ag_sensitivity_scorer = make_scorer(name='SE',
                                 score_func=sensitivity,
                                 optimum=1,
                                 greater_is_better=True)

    ag_specificity_scorer = make_scorer(name='SP',
                                 score_func=specificity,
                                 optimum=1,
                                 greater_is_better=True)
    
  
    tn_scorer = make_scorer(name='TN',score_func=tn_func,greater_is_better=True)

    
      
    fp_scorer = make_scorer(name='FP',score_func=fp_func,greater_is_better=False)

    
      
    fn_scorer = make_scorer(name='FN',score_func=fn_func,greater_is_better=False)

    

    tp_scorer = make_scorer(name='TP',score_func=tp_func,greater_is_better=True)

    
    # Append all metrics to list
    metrics_ls.append(tp_scorer)
    metrics_ls.extend((fp_scorer, fn_scorer, tn_scorer, ag_sensitivity_scorer,
                      ag_specificity_scorer, ag_accuracy_scorer, ag_mcc_scorer))
#     print(metrics_ls)
    return metrics_ls
#======================================================================================#
def AutoGluon(training_set,test_set,save_path,model_results,model_name):
    print('\nSETTING UP DATA FOR MODEL...')
    dat_train = pd.read_csv(training_set, sep=',')
    dat_train_smi = dat_train['SMILES'].tolist()
    print('\ndat_train no. of SMILES:')
    print(len(dat_train_smi))

    y_train = dat_train[['Binary Activity']]
    X_train,error_idx_ls = get_Morgan_fingerprint(dat_train_smi,2048,2)

    y_train = y_train.drop(error_idx_ls)
    X_train = X_train.drop(error_idx_ls)

    dat_test = pd.read_csv(test_set, sep=',')
    dat_test_smi = dat_test['SMILES'].tolist()
    print('\ndat_test no. of SMILES:')
    print(len(dat_test_smi))

    y_test = dat_test[['Binary Activity']]
    X_test,error_idx_ls = get_Morgan_fingerprint(dat_test_smi,2048,2)

    # Drop rows with errors
    y_test = y_test.drop(error_idx_ls)
    X_test = X_test.drop(error_idx_ls)

    print(X_train)
    print(y_train)

    print('\nDATA SET UP FOR MODEL')
    
    print('\nPREPARING AND TRAINING MODELS...')
    # The dataframes need to be converted to the object used byu the AutoGluon package
    label = 'Binary Activity'
    train_data = TabularDataset(pd.concat([X_train,y_train],axis=1))
    
    # This the maximum time limit in seconds for each set of models
    # Essentially, this means the models will take an estimated (13 * time_limit * (num_stack_levels +1)) 
    # amount of seconds to train. Note training time is also affected by the preset quality
    time_limit = 60 * 60
    metric = 'accuracy'
    
    # Best quality focuses on acc but takes longer to run. For more details see the package documentation
    preset = 'best_quality'
    
    # add hyperparameter_tune_kwargs if necessary
#     num_trials = 25  # try at most n different hyperparameter configurations for each type of model
#     search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

#     hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
#         'num_trials': num_trials,
#         'scheduler' : 'local',
#         'searcher': search_strategy,
#     }
    
    
    # Model ensembling with stacking/bagging
    # According to the documentation, "beyond hyperparameter-tuning with a correctly-specified evaluation metric,
    # two other methods to boost predictive performance are bagging and stack-ensembling. 
    # You’ll often see performance improve if you specify num_bag_folds = 5-10, 
    # num_stack_levels = 1-3 in the call to fit(), but this will increase training times and memory/disk usage."
    # By default no hyperparameter optimisation is done ie. when hyperparameter_tune_kwargs is not specified
    predictor = TabularPredictor(label, eval_metric=metric,path=model_name).fit(train_data,num_bag_folds=5, 
                                                                num_bag_sets=1, num_stack_levels=1, 
                                                                time_limit=time_limit,
                                                                presets=preset
#                                                                ,hyperparameter_tune_kwargs=hyperparameter_tune_kwargs
                                                               )
                                                               
    print('\nMODELS TRAINED')
    
    print('\nEVALUATING MODELS...')
    test_data = TabularDataset(pd.concat([X_test,y_test],axis=1))
    y_test = test_data[label]  # values to predict
    X_test = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
    X_test.head()
    
    y_pred = predictor.predict(X_test)
    #print("Predictions:  \n", y_pred)
    #perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
    
    #print("TRUE VALUES:  \n", y_test)
    y_test = y_test.tolist()
    y_pred = y_pred.tolist()
    
    add_metrics = create_AutoGluon_extra_metrics(y_test,y_pred)

    results_df = predictor.leaderboard(test_data, silent=True,
                                       extra_metrics=add_metrics
                                      )
    results_df.to_csv(model_results)
    print(results_df)
    print('\nMODELS EVALUATED')   
    
    return

def set_up_test_data(test_path):
    print('\nSETTING UP DATA FOR MODEL...')

    dat_test = pd.read_csv(test_path, sep=',')
    dat_test_smi = dat_test['SMILES'].tolist()
    print('\ndat_test no. of SMILES:')
    print(len(dat_test_smi))

    y_test = dat_test[['Binary Activity']]
    X_test,error_idx_ls = get_Morgan_fingerprint(dat_test_smi,2048,2)

    # Drop rows with errors
    y_test = y_test.drop(error_idx_ls)
    
    X_test = X_test.drop(error_idx_ls)

    print('\nDATA SET UP FOR MODEL')
    
    return X_test,y_test
    


#==============================================================================================#
#==============================================================================================#
# Set filepaths for functions

# Filename of model to load'
# model_root_desc = 'V2.0.1 Unified data'
model_root_desc = 'V2.0.1 Developmental toxicity'
# model_root_desc = 'V2.0.1 Reproductive toxicity'

# Filepath of test csv containing two columsn 'SMILES' and 'Binary Activity'
root_desc = 'V2.0.1 Reproductive toxicity'
# root_desc = 'V2.0.1 Developmental toxicity'

root = 'C:/Users/mwhw3/Desktop/DART project/'
# test_path = root + root_desc + '.csv'

total_runs = 5

overall_save = root + 'Transfer learning/' + root_desc + ' ' + model_root_desc + ' dev_to_repro_test_2 ' + \
str(total_runs) +' runs.csv'
#overall_save = root + 'AutoML models combined results/' + root_desc + ' ' + str(total_runs) +' runs.csv'

for run in range(1,total_runs+1):
    
    print('\n#=========================================================================================#')
    print('#=========================================================================================#')
    print('\n                               NOW PERFORMING RUN {}\n'.format(run))
    print('#=========================================================================================#')
    print('#=========================================================================================#')
    
    desc = root_desc + ' ' + str(run)
    model_desc = model_root_desc + ' ' + str(run)
    
    # Load location for models
    model_name = root + 'AutoML models/' + model_desc + '/' + 'Models/'
    print(model_name)

    # Save location for models results
    check_path = root + 'Transfer learning/' + desc + '/' + 'Results/'
    if os.path.exists(check_path)== False:
        os.makedirs(check_path)
        
    model_results = root + 'Transfer learning/' + desc + '/' + 'Results/' + desc + ' model results_2.csv'
    
    test_path = root + 'AutoML models/' + root_desc + ' ' + str(run) + '/Data/' + \
    root_desc + ' ' + str(run) + ' CSV test.csv'
    print(test_path)
    
    # Set up test data
    X_test,y_test = set_up_test_data(test_path)
    test_data = TabularDataset(pd.concat([X_test,y_test],axis=1))
    y_test = y_test['Binary Activity'].tolist()
    
    #==============================================================================================#                             

    # Load predictor   
    predictor = TabularPredictor.load(model_name)
    y_pred = predictor.predict(X_test)
    
    y_pred = y_pred.tolist()
    
    add_metrics = create_AutoGluon_extra_metrics(y_test,y_pred)

    results_df = predictor.leaderboard(test_data, silent=True,
                                       extra_metrics=add_metrics
                                      )
    results_df.to_csv(model_results)
    print(results_df)
    print('\nMODELS EVALUATED')
    
    
print('\n#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')
print('\n                         ALL MODELS EVALUATED WITH TEST SET(S)\n'                          )
print('#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')


#==============================================================================================#
#==============================================================================================#    
# Combine results for models across all runs    

# Filepath of csv containing two columsn 'SMILES' and 'Binary Activity'



#col_ls = ['TP','FP','FN','TN','SE','SP','Acc','MCC']
col_ls = ['model','SE','SP','Acc','MCC']

result_dict = {}
result_dict.clear()
for run in range(1,total_runs+1):
    desc = root_desc + ' ' + str(run)
    model_results = root + 'Transfer learning/' + desc + '/' + 'Results/' + desc + ' model results.csv'

    results_df = pd.read_csv(model_results)

    results_df = results_df[col_ls]
    #print(results_df)
    results_name = str(results_df) + str(run)
    result_dict[results_name] = results_df

    
combined_df = pd.concat(result_dict.values(),ignore_index=True)


mean_df = combined_df.groupby(by=['model']).mean()

print('\n#=========================================================================================#')
print('\n                                 DATAFRAME OF MEANS\n'                                      )
print('#=========================================================================================#\n')

mean_df['SE'] = 100 * mean_df['SE']
mean_df['SP'] = 100 * mean_df['SP']
mean_df['Acc'] = 100 * mean_df['Acc']

mean_df = mean_df.round({'SE': 1,'SP': 1,'Acc': 1,'MCC': 3}).astype(str)
print(mean_df)

std_df = combined_df.groupby(by=['model']).agg(np.std)
print('\n#=========================================================================================#')
print('\n                                 DATAFRAME OF STD\n'                                        )
print('#=========================================================================================#\n')

std_df['SE'] = 100 * std_df['SE']
std_df['SP'] = 100 * std_df['SP']
std_df['Acc'] = 100 * std_df['Acc']

std_df = std_df.round({'SE': 1,'SP': 1,'Acc': 1,'MCC': 3}).astype(str)
print(std_df)

# mean_df = mean_df.applymap(str)
# print(mean_df)
# std_df = std_df.applymap(str)
overall_df = mean_df

overall_df['SE'] = mean_df['SE'] + ' '+ str(u"\u00B1") + ' ' + std_df['SE']
overall_df['SP'] = mean_df['SP'] + ' '+ str(u"\u00B1") + ' ' + std_df['SP']
overall_df['Acc'] = mean_df['Acc'] + ' '+ str(u"\u00B1") + ' ' + std_df['Acc']
overall_df['MCC'] = mean_df['MCC'] + ' '+ str(u"\u00B1") + ' ' + std_df['MCC']

print('\n#=========================================================================================#')
print('\n                                 OVERALL DATAFRAME\n'                                        )
print('#=========================================================================================#\n')

overall_df.to_csv(overall_save)
print(overall_df)

print('\nFINISHED')    


#=========================================================================================#
#=========================================================================================#

                               NOW PERFORMING RUN 1

#=========================================================================================#
#=========================================================================================#
C:/Users/mwhw3/Desktop/DART project/AutoML models/V2.0.1 Developmental toxicity 1/Models/
C:/Users/mwhw3/Desktop/DART project/AutoML models/V2.0.1 Reproductive toxicity 1/Data/V2.0.1 Reproductive toxicity 1 CSV test.csv

SETTING UP DATA FOR MODEL...

dat_test no. of SMILES:
322
322

DATA SET UP FOR MODEL
                      model  score_test   TP  FP  FN   TN        SE        SP  \
0   RandomForestEntr_BAG_L2    0.782609  146  46  24  106  0.858824  0.697368   
1           CatBoost_BAG_L2    0.782609  147  47  23  105  0.864706  0.690789   
2   RandomForestGini_BAG_L2 

# For loading choice of models on a set of new chemicals

This code will load previously saved models and apply them to new chemicals (SMILES). The results can be returned as a dataframe containing the predicted asctivities/toxicity or as a file.

In [ ]:
# Code for using trained models to predict on an external test set or external data

# Author: Marcus Wei How Wang
# 5 May 2022

#======================================================================================#
import pandas as pd
import numpy as np

import os
import os.path
from os.path import isfile

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import make_scorer

from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

import pickle
from random import randrange

from autogluon.tabular import TabularDataset, TabularPredictor

#======================================================================================#
# This function gets the Morgan fingeprint given a SMILES dataframe and specified fingerprint parameters
def get_Morgan_fingerprint(smiles,nBits,fingerprint_radius):
    
    '''smiles dataframe'''
    error_idx_ls =[]
    error_idx_ls.clear()
    
    rdkit_molecules=[Chem.MolFromSmiles(x) for x in smiles]
    print(len(rdkit_molecules))
    rdkit_fingerprint=[]
    count = 0
    fingerprint_length = int(nBits)
    #print(rdkit_molecules[:1])
    for mol in rdkit_molecules:
        # if count % 1000 == 0:
        #     print('Now fingerprinting {} of {}'.format(count,len(rdkit_molecules)))
        bit_info={}
        try:
            fp=rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=fingerprint_radius, nBits=fingerprint_length,
                                                          bitInfo=bit_info)
        except:
            # get indexes of errors with errors
            error_idx_ls.append(rdkit_molecules.index(mol))

        rdkit_fingerprint.append(fp)
        count += 1
        
    fingerprint_df=pd.DataFrame([np.array(list(x)).astype(int) for x in rdkit_fingerprint])
    #fingerprint_df = pd.DataFrame(rdkit_fingerprint,columns=['BV'])
    
    return fingerprint_df, error_idx_ls

def get_MACCS_fingerprint(smiles):
    
    
    
    return fingerprint_df


# This function is expecting a input csv with two columns 'SMILES' and 'Binary Activity'
def split_train_test(csv,test_ratio,train_save,test_save,overwrite):
    
    # Read input and shuffle randomly
    input_df = pd.read_csv(csv)
    print(input_df)
    input_df = input_df.sample(frac=1)
    input_np = input_df.to_numpy()
    
    # split df randomly according to specified ratio
    train_df, test_df = train_test_split(input_np, test_size=test_ratio)
    
    train_df = pd.DataFrame(train_df,columns=['SMILES','Binary Activity'])
    test_df = pd.DataFrame(test_df,columns=['SMILES','Binary Activity'])
    
    # Save files
    # Ovewrite existing file present in folder
#     print(isfile(train_save))
#     print(isfile(test_save))
    if overwrite == False and isfile(train_save) == True:
        train_save = train_save[:-4] + str(randrange(100)) + train_save[-4:]
        train_df.to_csv(train_save)        
    else:
        train_df.to_csv(train_save)
    
    if overwrite == False and isfile(test_save) == True:
        test_save = test_save[:-4] + str(randrange(100)) + test_save[-4:]
        test_df.to_csv(test_save)        
    else:
        test_df.to_csv(test_save)
    
    
    return train_save,test_save

def most_probable_class(column_list):
    col_val = set(column_list)
    most_prob_class_count = 0
    most_prob_class = 0
    for ele in col_val:
        class_count = column_list.count(ele)
        if class_count > most_prob_class_count:
            most_prob_class_count = class_count
            most_prob_class = ele
            
    return most_prob_class


def calc_baseline_acc(df_column_list):
    
    # Baseline accuracy is the accuracy when all the predicted classes are the most probable class
    baseline_class = most_probable_class(df_column_list)
    baseline_acc = df_column_list.count(baseline_class) / len(df_column_list) * 100
    
    print('Baseline accuracy:')
    print(baseline_acc)
    
    return baseline_acc

def acc(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return (tp+tn)/(tp+fp+fn+tn)

def sensitivity(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return (tp)/(tp+fn)

def specificity(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return (tn)/(fp+tn)

def tp_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return tp

def fn_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    # Needs the - sign since predictor.leaderboard flips the sign
    return -fn  

def fp_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    # Needs the - sign since predictor.leaderboard flips the sign
    return -fp  

def tn_func(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    return tn  


def create_AutoGluon_extra_metrics(y_test,y_pred):
    from autogluon.core.metrics import make_scorer
    # Score functions need to have the definition:
    # score_func(y, y_pred, **kwargs)
    # Otherwise the calculations of the metrics will fail
    
    metrics_ls = []
    metrics_ls.clear()
    
    # Calculate confusion matrix for custom metrics
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    MCC = matthews_corrcoef(y_test, y_pred)
    
      
    # Define custom scoring functions to pass to predictor.leaderboard()
    ag_accuracy_scorer = make_scorer(name='Acc',
                                 score_func=acc,
                                 optimum=1,
                                 greater_is_better=True)
    
    ag_mcc_scorer = make_scorer(name='MCC',
                                 score_func=matthews_corrcoef,
                                 optimum=1,
                                 greater_is_better=True)
    
    ag_sensitivity_scorer = make_scorer(name='SE',
                                 score_func=sensitivity,
                                 optimum=1,
                                 greater_is_better=True)

    ag_specificity_scorer = make_scorer(name='SP',
                                 score_func=specificity,
                                 optimum=1,
                                 greater_is_better=True)
    
  
    tn_scorer = make_scorer(name='TN',score_func=tn_func,greater_is_better=True)

    
      
    fp_scorer = make_scorer(name='FP',score_func=fp_func,greater_is_better=False)

    
      
    fn_scorer = make_scorer(name='FN',score_func=fn_func,greater_is_better=False)

    

    tp_scorer = make_scorer(name='TP',score_func=tp_func,greater_is_better=True)

    
    # Append all metrics to list
    metrics_ls.append(tp_scorer)
    metrics_ls.extend((fp_scorer, fn_scorer, tn_scorer, ag_sensitivity_scorer,
                      ag_specificity_scorer, ag_accuracy_scorer, ag_mcc_scorer))
#     print(metrics_ls)
    return metrics_ls
#======================================================================================#
def AutoGluon(training_set,test_set,save_path,model_results,model_name):
    print('\nSETTING UP DATA FOR MODEL...')
    dat_train = pd.read_csv(training_set, sep=',')
    dat_train_smi = dat_train['SMILES'].tolist()
    print('\ndat_train no. of SMILES:')
    print(len(dat_train_smi))

    y_train = dat_train[['Binary Activity']]
    X_train,error_idx_ls = get_Morgan_fingerprint(dat_train_smi,2048,2)

    y_train = y_train.drop(error_idx_ls)
    X_train = X_train.drop(error_idx_ls)

    dat_test = pd.read_csv(test_set, sep=',')
    dat_test_smi = dat_test['SMILES'].tolist()
    print('\ndat_test no. of SMILES:')
    print(len(dat_test_smi))

    y_test = dat_test[['Binary Activity']]
    X_test,error_idx_ls = get_Morgan_fingerprint(dat_test_smi,2048,2)

    # Drop rows with errors
    y_test = y_test.drop(error_idx_ls)
    X_test = X_test.drop(error_idx_ls)

    print(X_train)
    print(y_train)

    print('\nDATA SET UP FOR MODEL')
    
    print('\nPREPARING AND TRAINING MODELS...')
    # The dataframes need to be converted to the object used byu the AutoGluon package
    label = 'Binary Activity'
    train_data = TabularDataset(pd.concat([X_train,y_train],axis=1))
    
    # This the maximum time limit in seconds for each set of models
    # Essentially, this means the models will take an estimated (13 * time_limit * (num_stack_levels +1)) 
    # amount of seconds to train. Note training time is also affected by the preset quality
    time_limit = 60 * 60
    metric = 'accuracy'
    
    # Best quality focuses on acc but takes longer to run. For more details see the package documentation
    preset = 'best_quality'
    
    # add hyperparameter_tune_kwargs if necessary
#     num_trials = 25  # try at most n different hyperparameter configurations for each type of model
#     search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

#     hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
#         'num_trials': num_trials,
#         'scheduler' : 'local',
#         'searcher': search_strategy,
#     }
    
    
    # Model ensembling with stacking/bagging
    # According to the documentation, "beyond hyperparameter-tuning with a correctly-specified evaluation metric,
    # two other methods to boost predictive performance are bagging and stack-ensembling. 
    # You’ll often see performance improve if you specify num_bag_folds = 5-10, 
    # num_stack_levels = 1-3 in the call to fit(), but this will increase training times and memory/disk usage."
    # By default no hyperparameter optimisation is done ie. when hyperparameter_tune_kwargs is not specified
    predictor = TabularPredictor(label, eval_metric=metric,path=model_name).fit(train_data,num_bag_folds=5, 
                                                                num_bag_sets=1, num_stack_levels=1, 
                                                                time_limit=time_limit,
                                                                presets=preset
#                                                                ,hyperparameter_tune_kwargs=hyperparameter_tune_kwargs
                                                               )
                                                               
    print('\nMODELS TRAINED')
    
    print('\nEVALUATING MODELS...')
    test_data = TabularDataset(pd.concat([X_test,y_test],axis=1))
    y_test = test_data[label]  # values to predict
    X_test = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
    X_test.head()
    
    y_pred = predictor.predict(X_test)
    #print("Predictions:  \n", y_pred)
    #perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
    
    #print("TRUE VALUES:  \n", y_test)
    y_test = y_test.tolist()
    y_pred = y_pred.tolist()
    
    add_metrics = create_AutoGluon_extra_metrics(y_test,y_pred)

    results_df = predictor.leaderboard(test_data, silent=True,
                                       extra_metrics=add_metrics
                                      )
    results_df.to_csv(model_results)
    print(results_df)
    print('\nMODELS EVALUATED')   
    
    return

def set_up_test_data(test_path):
    print('\nSETTING UP DATA FOR MODEL...')

    dat_test = pd.read_csv(test_path, sep=',')
    dat_test_smi = dat_test['SMILES'].tolist()
    print('\ndat_test no. of SMILES:')
    print(len(dat_test_smi))

    y_test = dat_test[['Binary Activity']]
    X_test,error_idx_ls = get_Morgan_fingerprint(dat_test_smi,2048,2)

    # Drop rows with errors
    y_test = y_test.drop(error_idx_ls)
    
    X_test = X_test.drop(error_idx_ls)

    print('\nDATA SET UP FOR MODEL')
    
    return X_test,y_test
    
def set_up_interpreting_results(test_path,y_pred,run):
    print('\nSETTING UP RESULTS FOR INTERPRETATION...')

    dat_test = pd.read_csv(test_path, sep=',')
    dat_test_smi = dat_test['SMILES'].tolist()
    print('\ndat_test no. of SMILES:')
    print(len(dat_test_smi))

    interpret_df = dat_test[['SMILES']]
    interpret_df = interpret_df.join(dat_test['Binary Activity'])
    interpret_df['y_pred'] = y_pred

    # Add columns determining which are misclassified
    # False positives
    FP_ls = []
    FP_ls.clear()
    
    for x in range(0,len(interpret_df)):
        if interpret_df.iloc[x]['Binary Activity'] == 0 and interpret_df.iloc[x]['y_pred'] == 1:
            FP_ls.append("True")
        else:
            FP_ls.append("False")
            
    # False negatives
    FN_ls = []
    FN_ls.clear()
    
    for x in range(0,len(interpret_df)):
        if interpret_df.iloc[x]['Binary Activity'] == 1 and interpret_df.iloc[x]['y_pred'] == 0:
            FN_ls.append("True")
        else:
            FN_ls.append("False")    

    interpret_df['False positive'] = FP_ls
    interpret_df['False negative'] = FN_ls
    
    interpret_df['run_no'] = run
            
    print('\nRESULTS SET UP FOR INTERPRETATION')
    
    return interpret_df

def predict_new_chemicals(test_path):
    print('\nSETTING UP DATA FOR MODEL...')

    dat_test = pd.read_csv(test_path, sep=',')
    dat_test_smi = dat_test['SMILES'].tolist()
    print('\ndat_test no. of SMILES:')
    print(len(dat_test_smi))

    X_test,error_idx_ls = get_Morgan_fingerprint(dat_test_smi,2048,2)

    # Drop rows with errors
  
    X_test = X_test.drop(error_idx_ls)

    print('\nDATA SET UP FOR MODEL')
    
    return X_test
#==============================================================================================#
#==============================================================================================#
# Set filepaths for functions

# Filename of model to load'
# model_root_desc = 'V2.0.1 Developmental toxicity'
model_root_desc = 'V2.0.1 Reproductive toxicity'

# Filepath of test csv containing two columsn 'SMILES' and 'Binary Activity'
root = 'C:/Users/mwhw3/Desktop/DART project/'

# Set number of runs to consider
# For each run, there will be a set of trained models.
# If only making a single prediction, just use any of the runs since model performance is rather consistent across all runs
total_runs = 1

# Set filepath for overall save if doing additional processing later
# overall_save = root + 'AutoML models combined interpreting predicted results/' + model_root_desc + \
# str(total_runs) +' runs V2 interpreting predicted results.csv'
#overall_save = root + 'AutoML models combined results/' + root_desc + ' ' + str(total_runs) +' runs.csv'

#==============================================================================================#
#==============================================================================================#


for run in range(1,total_runs+1):
    
    print('\n#=========================================================================================#')
    print('#=========================================================================================#')
    print('\n                               NOW PERFORMING RUN {}\n'.format(run))
    print('#=========================================================================================#')
    print('#=========================================================================================#')
    
    model_desc = model_root_desc + ' ' + str(run)
    dataset_desc = model_desc +  ' CSV test.csv'

    # Load location for models
    model_name = root + 'AutoML models/' + model_desc + '/' + 'Models/'
    print(model_name)

    # Save location for model results if necessary
#     check_path = root + 'Urgent chemicals/' + model_desc + '/'
#     if os.path.exists(check_path)== False:
#         os.makedirs(check_path)
        
#     model_results = root + 'AutoML models/' + model_desc + '/' + 'Interpreting Predicted Results/' + model_desc + ' interpreting predicted results.csv'
#     interpret_df_save = root + 'AutoML models/' + model_desc + '/' + 'Interpreting Predicted Results/' + model_desc + ' V2 interpreting predicted results.csv'
    
    #==============================================================================================#
    # Set up test data
    test_path = root + 'Urgent chemicals/' + 'DART_chemicals.csv'
    
    # If test data has 'Binary Activity'
#     X_test,y_test = set_up_test_data(test_path)

#     test_data = TabularDataset(pd.concat([X_test,y_test],axis=1))
#     y_test = y_test['Binary Activity'].tolist()
    
    # If test data does not have 'Binary Activity'
    X_test = predict_new_chemicals(test_path)
    #==============================================================================================#                             

    # Load predictor   
    predictor = TabularPredictor.load(model_name)
    all_models = predictor.get_model_names()
    #     print(all_models)
    
    # *** Set this manually ***
    # Choose the models you want to use
    # If not sure of the model name, refer to print(all_models)
    if model_root_desc == 'V2.0.1 Developmental toxicity':
        best_model = 'WeightedEnsemble_L2'
    if model_root_desc == 'V2.0.1 Reproductive toxicity':
        best_model = 'RandomForestGini_BAG_L2'
    if model_root_desc == 'V2.0.1 Unified data':
        best_model = 'ExtraTreesEntr_BAG_L2'
    
    ###
    # If only using one model to predict
    model_i = 0  # Set default index of model to use in set of models within predictor
    i = 0
    
    model_i = [i for i,x in enumerate(all_models) if best_model == x]
    model_i = model_i[0]
    model_to_use = predictor.get_model_names()[model_i]
    model_pred = predictor.predict(X_test, model=model_to_use)
    print("\nPredicting using %s model" % (model_to_use))               
    
    y_pred = predictor.predict(X_test)
    
    y_pred = y_pred.tolist()
    
    # Can save this df to csv if need be
    y_pred_df = pd.DataFrame({'Result':y_pred})
    print(y_pred_df)
    
    
#     interpret_df = set_up_interpreting_results(test_path,y_pred,run)
    
#     # This is to create a set of metrics. Only do this if your new chemicals have known activities/toxicities
#     add_metrics = create_AutoGluon_extra_metrics(y_test,y_pred)
#     results_df = predictor.leaderboard(test_data, silent=True,
#                                        extra_metrics=add_metrics
#                                       )      
    #results_df.to_csv(model_results)
    
    
#     interpret_df.to_csv(interpret_df_save)
#     print(results_df)
#     print(interpret_df)


    ###
#     # If using multiple models to predict ie. check prediction of ensemble of models

#     # This gets the models (names) of all the models used
#     all_models = predictor.get_model_names()

#     count = 0
#     for use_model in all_models:
#         y_pred = predictor.predict(X_test, model=use_model)
#         y_pred = y_pred.tolist()
#         if count == 0:
#             interpret_df = set_up_interpreting_results(test_path,y_pred,run)
#             combined_df = interpret_df
#             combined_df['Model'] = str(use_model)
#             count = count + 1
#             print(combined_df)
#         else:
#             interpret_df = set_up_interpreting_results(test_path,y_pred,run)
#             interpret_df['Model'] = str(use_model)
#             combined_df = combined_df.merge(interpret_df, how='inner', on='SMILES')
#             print(combined_df)
#             count = count + 1
#             if count == len(all_models):
#                 combined_df_save = root + 'AutoML models combined interpreting predicted results/' +  model_desc + ' V2 misclassified predicted results.csv'
#                 combined_df.to_csv(combined_df_save)

    print('\nMODELS EVALUATED')
    
    
print('\n#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')
print('\n                         ALL MODELS EVALUATED\n'                          )
print('#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')
print('#=========================================================================================#')
   

# For getting the "nearest neighbours" of the chemicals

This would get the most similar chemicals in the training set for the chemicals in the test set.

In [ ]:
# For calculating the average similarity per molecule for the 5 nearest neighbours in both the training and the test sets

import pandas as pd
import numpy as np


from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdDepictor

from rdkit import DataStructs

from statistics import mean

#===================================================================================#
def get_Morgan_fingerprint(smiles,nBits,fingerprint_radius):
    
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol,fingerprint_radius,nBits=nBits)
    
    return fp


#==============================================================================================#
def calc_Tanimoto_similarity(smiles1,smiles2):
    fp1 = get_Morgan_fingerprint(smiles1,2048,2)
    fp2 = get_Morgan_fingerprint(smiles2,2048,2)
    similarity = DataStructs.FingerprintSimilarity(fp1,fp2,metric=DataStructs.TanimotoSimilarity)
    
    return similarity

#===================================================================================#
# Set filenames for required data
# Ensure that you model_root_desc and the training data are for the same toxicity
total_runs = 1
root = 'C:/Users/mwhw3/Desktop/DART project/'
# model_root_desc = 'V2.0.1 Developmental toxicity'
model_root_desc = 'V2.0.1 Reproductive toxicity'
model_desc = model_root_desc + ' ' + str(total_runs)

# Input df. This contains the chemicals for which you want to find the most similar compounds for.
input_filename = root + 'Urgent chemicals/' +   \
'DART_chemicals.csv'

# Load model training data to calculate similarties with training set
# training_input_filename = root + 'AutoML models/' +  model_desc + \
# '/Data/V2.0.1 Developmental toxicity 1 CSV train.csv'
training_input_filename = root + 'AutoML models/' +  model_desc + \
'/Data/V2.0.1 Reproductive toxicity 1 CSV train.csv'


# kmeans_analysis_input = root + 'Feature plots/kmeans/' +  model_desc + ' V2 all K-means PCA misclassifion analysis.csv'

# final_analysis_save = root + 'Similarities/' +  model_desc + \
# ' V2 training and test similarity pair analysis.csv'

# merged_df_save = root + 'Similarities/' +  model_desc + \
# ' V2 merged training and test similarity pair analysis.csv'

# # Sometimes csv saves but columns load incorrectly when opening with excel
# # This mitigates the issue entierly by directly saving as xlsx
# merged_df_save_xlsx = root + 'Similarities/' +  model_desc + \
# ' V2 merged training and test similarity pair analysis.xlsx'
#===================================================================================#

initial_df = pd.read_csv(input_filename)

try:
    initial_df = initial_df[initial_df.columns.drop(list(initial_df.filter(regex='Unnamed')))]
except:
    pass

training_df = pd.read_csv(training_input_filename)

try:
    training_df = training_df[training_df.columns.drop(list(training_df.filter(regex='Unnamed')))]
except:
    pass
print('TRAINING DF\n')
print(training_df)

input_df = initial_df
test_smiles_df = input_df[['SMILES']]
training_smiles_df = training_df[['SMILES']]
print('INPUT DF\n')
print(input_df)

#===================================================================================#
# Get list of Tanimoto similarities per molecule in the dataset
smiles_df_ls = []

for x in range(0,len(test_smiles_df)):
    
    # If calculating between training and test set
    copy_df = training_smiles_df.copy()    
    
    # If calculating within test set
#     copy_df = test_smiles_df.copy()
#     copy_df = copy_df.drop([x], axis=0)
    
    # Calculate list of Tanimoto similarities
    # Note that this calculates for all similarity pairs so expect major time consumption here if running for
    # a large number of chemicals
    similarity_ls = []
    smiles1 = test_smiles_df.loc[x]['SMILES']
#     smiles1 = training_smiles_df.loc[x]['SMILES']
    for index, row in copy_df.iterrows():
        smiles2 = row['SMILES']
        similarity_ls.append(calc_Tanimoto_similarity(smiles1,smiles2))
        if len(similarity_ls) % 150 == 0 and x % 100 == 0:
            print('\nNOW PROCESSING INDEX {} IN COPY_DF FOR MOLECULE {}'.format(index,x))
            print(len(similarity_ls),'SIMILARITY PAIRS CALCULATED')
    smiles_df_ls.append(similarity_ls)
        
final_analysis_df = input_df.copy()
# final_analysis_df = training_smiles_df.copy()
final_analysis_df['similarity'] = smiles_df_ls
final_analysis_df = final_analysis_df[['SMILES','similarity']]

# Get average similarity and average of top 1,3,5 similarity per compound
average_sim = []
top_1 = []
top_3 = []
top_5 = []

for index, row in final_analysis_df.iterrows():
    to_process = row['similarity']
    to_process = sorted(to_process, reverse=True)
    
    average_sim.append(mean(to_process))
    top_1.append(to_process[0])
    top_3.append(mean(to_process[0:3]))
    top_5.append(mean(to_process[0:5]))
    
    
final_analysis_df['average_sim'] = average_sim
final_analysis_df['top_1'] = top_1
final_analysis_df['top_3'] = top_3
final_analysis_df['top_5'] = top_5

print(final_analysis_df)
# final_analysis_df.to_csv(final_analysis_save)

print('\nFINISHED')

# Process final_analysis_df if doing train test pairs to get top n similar structures in training set

In [ ]:
# Continued from previous cell. Set filenames
# Save to excel. Sometimes the csv will not load correctly so saving to excel prevents this issue.
sim_df_save_xlsx = root + 'Urgent chemicals/' +  model_desc + \
' similarity results.xlsx'

#===================================================================================================#

sim_df = final_analysis_df

# merged_smiles_ls = input_df['SMILES']
sim_smiles_ls = training_df['SMILES']
print(len(sim_smiles_ls))
sorted_SMILES = []
sorted_similarity = []
for index,row in final_analysis_df.iterrows():
    to_sort = row['similarity']
#     print(to_sort)
    # Get top 5 indices and values for similarity pairs
    # Change the right hand number if you require more than top 5 pairs
    sorted_tuple = sorted(enumerate(to_sort), reverse=True, key=lambda x: x[1])[:5]
#     print(sorted_tuple)
    indices,value = [list(ele) for ele in zip(*sorted_tuple)]
#     index = [index for index, value in sorted(enumerate(to_sort), reverse=True, key=lambda x: x[1])[:6]
             
    # Get corresponding SMILES as list
#     print(indices)
#     print(value)
    temp = [sim_smiles_ls[i] for i in indices]  
    sorted_SMILES.append(temp)
    sorted_similarity.append(value)
    
final_analysis_df['sorted_SMILES'] = sorted_SMILES
final_analysis_df['sorted_similarity'] = sorted_similarity
                   
print('\nDF PROCESSED')
print(sim_df)
sim_df.to_excel(sim_df_save_xlsx, index=True)

print('\nFINISHED')